In [1]:
%pip install pandas scikit-learn

from contextlib import contextmanager
from pathlib import Path
from timeit import default_timer

from numpy import float64, uint8

from pandas import DataFrame, Series

from sklearn.datasets import fetch_openml
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

Note: you may need to restart the kernel to use updated packages.


In [2]:
@contextmanager
def timer():
  '''from given starter code'''
  
  start = default_timer()
  def elapser(): return default_timer() - start
  yield lambda: elapser()
  end = default_timer()
  def elapser(): return end - start

In [3]:
if Path('sgd_time.txt').is_file():
  with open('sgd_time.txt', 'r') as sgd_time:
    print(sgd_time.read())
else:
  mnist: DataFrame = fetch_openml('mnist_784', version=1)

  X: DataFrame = mnist['data'].astype(float64)
  y: Series = mnist['target'].astype(uint8)

  X_train: DataFrame
  X_test: DataFrame 
  X_train, X_test = X[:60000], X[60000:]

  y_train: Series
  y_test: Series
  y_train, y_test = y[:60000], y[60000:]

  scaler: StandardScaler = StandardScaler()
  X_train_scale : DataFrame = scaler.fit_transform(X_train)
  X_test_scale : DataFrame = scaler.transform(X_test)

  sgd: SGDClassifier = SGDClassifier(random_state=42)
  with timer() as sgd_timer:
    sgd.fit(X_train_scale, y_train)
  
  print(sgd_timer)
  with open('sgd_time.txt', 'w') as sgd_time:
    sgd_time.write(sgd_timer)

KeyboardInterrupt: 